In [1]:
using PerlaTonettiWaugh

# Steady-State Comparison

In [2]:
params = load_parameters("parameters/calibration_params.csv")
baseline = merge(params, (d = params.d_0,))
baseline_sol = stationary_algebraic(baseline, settings_defaults());

In [3]:
new_params = merge(params, (d = params.d_T,))
new_sol = stationary_algebraic(new_params, settings_defaults());

In [4]:
@show (new_sol.g, baseline_sol.g);

(new_sol.g, baseline_sol.g) = (0.010250822794504864, 0.007913401963163308)


In [5]:
@show (1 - new_sol.λ_ii, 1 - baseline_sol.λ_ii);

(1 - new_sol.λ_ii, 1 - baseline_sol.λ_ii) = (0.14442864132703293, 0.10629127170507902)


In [6]:
@show (new_sol.U_bar, baseline_sol.U_bar);

(new_sol.U_bar, baseline_sol.U_bar) = (17.534340776669858, 12.322561075850476)


In [7]:
@show 100*consumption_equivalent(new_sol.U_bar, baseline_sol.U_bar, params);

100 * consumption_equivalent(new_sol.U_bar, baseline_sol.U_bar, params) = 11.181899506569803


# Transition Dynamics

In [8]:
using CSV, Plots, JSON, BenchmarkTools, Parameters, DataFrames, Suppressor
gr(fmt = :png)

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260


Plots.GRBackend()

In [9]:
overall_parameters = load_parameters("parameters/calibration_params.csv")

(ρ = 0.020338044668517, σ = 3.16692413583811, θ = 4.98897658793826, κ = 0.104196324793307, χ = 0.126846612050694, μ = -0.0310646242175711, υ = 0.0483011406016648, ζ = 1, δ = 0.02, N = 10, γ = 1, η = 0, Theta = 1, d_0 = 3.0224928254626, d_T = 2.82024354291634)

In [10]:
settings = settings_defaults(); # numerical solution settings
@unpack stationary_T, stationary_0, change_welfare, change_trade = compare_steady_states(overall_parameters, settings);

Note that everything from here on out is using the **numerical** solution, so the values might be slightly different than the algebraic one. 

We do this so that the endpoints are valid targets for the dynamical experiment. 

To use the algebraic one, simply add `algebraic = true` to the `compare_steady_states` function above. 

In [11]:
@show (stationary_T.g, stationary_T.z_hat, stationary_T.Ω, stationary_T.L_tilde); # inspect time T solution

(stationary_T.g, stationary_T.z_hat, stationary_T.Ω, stationary_T.L_tilde) = (0.010256161538647203, 1.8523062408854238, 0.6508139010470979, 0.24606267582490407)


In [12]:
@show (stationary_0.g, stationary_0.z_hat, stationary_0.Ω, stationary_0.L_tilde); # inspect time 0 solution

(stationary_0.g, stationary_0.z_hat, stationary_0.Ω, stationary_0.L_tilde) = (0.007920170474471236, 1.988197541033372, 0.6852422606330657, 0.24226457898689088)


In [13]:
cachename = model_cachename(overall_parameters, settings) # uniquely identify the run by parameters + settings
cachepath = joinpath(pwd(), "data", cachename * ".csv")
use_cache = true # change this to false if you don't want to use the cache for a run
write_cache = true # change this to false if you don't want to generate *any* cache files for a run

true

In [14]:
cachename # this is the hash for this run

"1921969888308317018"

In [15]:
if isfile(cachepath) && use_cache # read if possible
    println("using cache.")
    solved = CSV.read(cachepath);
else
    println("solving without cache.")
    @time solution = @suppress solve_transition(overall_parameters, settings);
    solved = solution.results 
end

solving without cache.
618.985817 seconds (2.11 G allocations: 720.198 GiB, 28.47% gc time)


,t,g,z_hat,Ω,E,v_1,L_tilde,entry_residual
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-20.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,2.08367e-12
2,-15.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,2.08367e-12
3,-10.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,2.08367e-12
4,-5.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,2.08367e-12
5,-1.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,2.08367e-12
6,0.0,0.00903988,1.87401,0.685242,0.0156134,6.86678,0.229549,0.0
7,0.25,0.00904654,1.87389,0.684969,0.0156577,6.86678,0.229729,0.0
8,0.5,0.0090532,1.87377,0.684697,0.0157018,6.86678,0.229908,0.0
9,0.75,0.00905986,1.87364,0.684425,0.0157457,6.86678,0.230085,0.0


In [16]:
if write_cache
    mkpath(joinpath(pwd(), "data")); # if this doesn't eist for whatever reason
    CSV.write(cachepath, solved); # write DataFrame
    write(joinpath(pwd(), "data", cachename * ".json"), JSON.json((parameters = overall_parameters, settings = merge(settings, (interp = typeof(settings.interp), stationary_x0 = typeof(settings.stationary_x0), fixedpoint_x0 = typeof(settings.fixedpoint_x0) ))))); # write parameters
else
    # nothing
end

6322

## Welfare

Here's welfare inclusive of the transition path:

In [17]:
ce_transition = 100*consumption_equivalent(solved.U[1], stationary_0.U_bar, load_parameters("parameters/calibration_params.csv"));
@show ce_transition;

ce_transition = 0.0


Plots for the transition path can be found in the notebook [TransitionDynamicsFigures.ipynb](https://github.com/jlperla/PerlaTonettiWaugh.jl/blob/master/TransitionDynamicsFigures.ipynb)